<a href="https://colab.research.google.com/github/jdfergason/home-automation/blob/main/Google_Nest_API_thermostat_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example for making Nest API calls

Please start by making a copy of this notebook.

## Enter your credentials
Enter your credentials and run the cell to generate a login URL. Click the URL and log in to your Google account.

In [3]:
project_id = 'f07a2511-46a0-4e99-aca3-38c1fa901061'
client_id = '536634890189-2f8md4aug21v5eu3uacvgju15kn33d1c.apps.googleusercontent.com'
client_secret = 'KSAr1lny4nrHFs3GHgdu7-Sk'
redirect_uri = 'http://automate.corwynn.com:8000/auth'

url = 'https://nestservices.google.com/partnerconnections/'+project_id+'/auth?redirect_uri='+redirect_uri+'&access_type=offline&prompt=consent&client_id='+client_id+'&response_type=code&scope=https://www.googleapis.com/auth/sdm.service'
print("Go to this URL to log in:")
print(url)

Go to this URL to log in:
https://nestservices.google.com/partnerconnections/f07a2511-46a0-4e99-aca3-38c1fa901061/auth?redirect_uri=http://automate.corwynn.com:8000/auth&access_type=offline&prompt=consent&client_id=536634890189-2f8md4aug21v5eu3uacvgju15kn33d1c.apps.googleusercontent.com&response_type=code&scope=https://www.googleapis.com/auth/sdm.service


After logging in you are sent to URL you specified as redirect_url. Google added a query to end that looks like this: ?code=.....&scope=...
Copy the part between code= and &scope= and add it below:

In [14]:
code = '4/0AY0e-g5hYjRmM8-wL8Cs3qLAKkTXn3oVLVpa3ED_Y9vD7NSbMtwqOuH9v1sCsKhccH4_Bg'

## Get tokens
Now we can use this code to retrieve an access token and a refresh token:

In [15]:
# Get tokens

import requests

params = (
    ('client_id', client_id),
    ('client_secret', client_secret),
    ('code', code),
    ('grant_type', 'authorization_code'),
    ('redirect_uri', redirect_uri),
)

response = requests.post('https://www.googleapis.com/oauth2/v4/token', params=params)

response_json = response.json()
access_token = response_json['token_type'] + ' ' + str(response_json['access_token'])
print('Access token: ' + access_token)
refresh_token = response_json['refresh_token']
print('Refresh token: ' + refresh_token)

Access token: Bearer ya29.a0AfH6SMB-DOKNtfBK6QXk94jiLtGDUPObuK-Bom2HJqMqQwrlY6YXKCMFw35QdW3GmAp4B-g6P2z-_e7tiEQVOYWc1u-Y1w2dGtSLosOUGthq-Y6daKN-4B8s21OvlVwjXURWtZxzrozigAUi98wMoweUaTS7cRnuvqLXwl94EEw
Refresh token: 1//06sT5X3hG0dhKCgYIARAAGAYSNwF-L9IrRpkPSUoDUGJEfsWwMSvnmk5WP2OWhGEi91iqS6U5zYouA3lx8MLXREQldCiZUTUj3io


## Refresh access token
The access token is only valid for 60 minutes. You can use the refresh token to renew it.

In [6]:
# Refresh token

params = (
    ('client_id', client_id),
    ('client_secret', client_secret),
    ('refresh_token', refresh_token),
    ('grant_type', 'refresh_token'),
)

response = requests.post('https://www.googleapis.com/oauth2/v4/token', params=params)

response_json = response.json()
access_token = response_json['token_type'] + ' ' + response_json['access_token']
print('Access token: ' + access_token)

Access token: Bearer ya29.a0AfH6SMA5hEODr2YM7H5oucYN4zezaL_sKZb772a_JsTBlGAB7gfzxmXvk7okVZapODLHmpXXwuaQ2IZyJc_OrMGFxN-yMd7n1CwshNDsg8qbcvRVLz5xVdbwwmhNpbrInkrL6uyts7XQsL2VVfuBOE57pwappfKYAgltpHTwFjU


## Get structures and devices
Now lets get some information about what devices we have access to and where these are "located". Devices are part of a structure (such as your home). We can get information about the structures we have access to:

In [16]:
# Get structures

url_structures = 'https://smartdevicemanagement.googleapis.com/v1/enterprises/' + project_id + '/structures'

headers = {
    'Content-Type': 'application/json',
    'Authorization': access_token,
}

response = requests.get(url_structures, headers=headers)

print(response.json())

{'structures': [{'name': 'enterprises/f07a2511-46a0-4e99-aca3-38c1fa901061/structures/AVPHwEuzrSjSge3q_TsajMLdVPjq-MceRT3XEei7yDJ2rv7MGoTBDMkx9UjQS59OjNAdLLtGBGpE-329byewm8EKIGhpew', 'traits': {'sdm.structures.traits.Info': {'customName': 'Home'}}}]}


But we can also directly retrieve the devices we have access to:

In [26]:
# Get devices

url_get_devices = 'https://smartdevicemanagement.googleapis.com/v1/enterprises/' + project_id + '/devices'
# url_get_devices = 'https://smartdevicemanagement.googleapis.com/v1/enterprises/devices'

headers = {
    'Content-Type': 'application/json',
    'Authorization': access_token,
}

response = requests.get(url_get_devices, headers=headers)

print(response.json())

response_json = response.json()
device_0_name = response_json['devices'][0]['name']
# device_0_name = 'enterprises/f07a2511-46a0-4e99-aca3-38c1fa901061/structures/AVPHwEuzrSjSge3q_TsajMLdVPjq-MceRT3XEei7yDJ2rv7MGoTBDMkx9UjQS59OjNAdLLtGBGpE-329byewm8EKIGhpew'
print(device_0_name)

{}


KeyError: ignored

## Get device stats
For this example I simply took the first item of the array of devices. I assume most people probably have one Nest thermostat anyway.

The name of a device can be used to retrieve data from this device and to send commands to it. Lets get soms stats first:

In [13]:
# Get device stats

url_get_device = 'https://smartdevicemanagement.googleapis.com/v1/' + device_0_name

headers = {
    'Content-Type': 'application/json',
    'Authorization': access_token,
}

response = requests.get(url_get_device, headers=headers)

response_json = response.json()

print(response_json)

humidity = response_json['traits']['sdm.devices.traits.Humidity']['ambientHumidityPercent']
print('Humidity:', humidity)
temperature = response_json['traits']['sdm.devices.traits.Temperature']['ambientTemperatureCelsius']
print('Temperature:', temperature)

{'name': 'enterprises/f07a2511-46a0-4e99-aca3-38c1fa901061/structures/AVPHwEuzrSjSge3q_TsajMLdVPjq-MceRT3XEei7yDJ2rv7MGoTBDMkx9UjQS59OjNAdLLtGBGpE-329byewm8EKIGhpew', 'traits': {'sdm.structures.traits.Info': {'customName': 'Home'}}}


KeyError: ignored

## Set thermostat to HEAT
And last but not least, lets send some commands to our thermostat. The cell below contains the code to set the mode to "HEAT":

In [ ]:
# Set mode to "HEAT"

url_set_mode = 'https://smartdevicemanagement.googleapis.com/v1/' + device_0_name + ':executeCommand'

headers = {
    'Content-Type': 'application/json',
    'Authorization': access_token,
}

data = '{ "command" : "sdm.devices.commands.ThermostatMode.SetMode", "params" : { "mode" : "HEAT" } }'

response = requests.post(url_set_mode, headers=headers, data=data)

print(response.json())

## Set a new temperature
And finally we can set a temperature by executing this command:

In [ ]:
set_temp_to = 21.0

In [ ]:
# Set temperature to set_temp_to degrees

url_set_mode = 'https://smartdevicemanagement.googleapis.com/v1/' + device_0_name + ':executeCommand'

headers = {
    'Content-Type': 'application/json',
    'Authorization': access_token,
}

data = '{"command" : "sdm.devices.commands.ThermostatTemperatureSetpoint.SetHeat", "params" : {"heatCelsius" : ' + str(set_temp_to) + '} }'

response = requests.post(url_set_mode, headers=headers, data=data)

print(response.json())